# И это ещё не всё!

Знания по темам этого модуля зачастую проверяют работодатели на собеседованиях. Предлагаем вам ознакомиться с вопросом, с которым вы можете столкнуться при поиске работы. 

Важно! Это задание на самопроверку, оно необязательное, и мы не будем оценивать качество его решения. Настоятельно рекомендуем вам попробовать найти ответ самостоятельно.

Имеются две базы данных (два листа Excel-файла): база с ценами конкурентов (Data_Parsing) и внутренняя база компании (Data_Company).

В базе парсинга есть два id, однозначно определяющие товар: producer_id и producer_color.

В базе компании есть два аналогичных поля: item_id и color_id.

Нам известно, что коды в двух базах отличаются наличием набора служебных символов. В базе парсинга встречаются следующие символы: _, -, ~, \\, /.

Необходимо:

Считать данные из Excel в DataFrame (Data_Parsing) и (Data_Company).
Подтянуть к базе парсинга данные из базы компании (item_id, color_id, current_price) и сформировать столбец разницы цен в % (цена конкурента к нашей цене).
Определить сильные отклонения от среднего в разности цен в пределах бренда-категории (то есть убрать случайные выбросы, сильно искажающие сравнение). Критерий — по вкусу, написать комментарий в коде.
Записать новый файл Excel с базой парсинга, приклееными к ней столбцами из пункта 2 и с учётом пункта 3 (можно добавить столбец outlier и проставить Yes для выбросов).

In [26]:
import pandas as pd
import numpy as np
#import openpyxl

# Считать данные из Excel в DataFrame
data_parsing = pd.read_excel('data/Data_TSUM.xlsx', sheet_name='Data_Parsing')
data_company = pd.read_excel('data/Data_TSUM.xlsx', sheet_name='Data_Company')


# Функция для очистки ID от указанных служебных символов
def clean_id(id):
    return ''.join(e for e in id if e not in ['_', '-', '~', '\\', '/'])

# Очистка ID в обоих DataFrame
data_parsing['clean_producer_id'] = data_parsing['producer_id'].apply(clean_id)
data_parsing['clean_producer_color'] = data_parsing['producer_color'].apply(clean_id)
data_company['clean_item_id'] = data_company['item_id'].apply(clean_id)
data_company['clean_color_id'] = data_company['color_id'].apply(clean_id)

# Объединение DataFrame по очищенным ID
merged_data = pd.merge(data_parsing, data_company, left_on=['clean_producer_id', 'clean_producer_color'], right_on=['clean_item_id', 'clean_color_id'], how='left')

# Расчет разницы цен в процентах
merged_data['price_difference_%'] = ((merged_data['price'] - merged_data['current price']) / merged_data['current price']) * 100

display(merged_data.head())

# Функция для определения выбросов на основе z-оценки
def identify_outliers(df, column):
    df['z_score'] = (df[column] - df[column].mean()) / df[column].std()
    df['outlier'] = np.where(np.abs(df['z_score']) > 2, 'Yes', 'No')  # Используем порог z-оценки 2 для определения выбросов
    return df

# # Применение функции определения выбросов в пределах каждой группы бренд-категории
# grouped = merged_data.groupby(['brand_x', 'Category_x'], group_keys=False, as_index=False)
# #grouped = merged_data.groupby(['brand_x', 'Category_x'], group_keys=False, as_index=False, include_groups=False)

# merged_data = grouped.apply(lambda x: identify_outliers(x, 'price_difference_%').drop(columns=['brand_x', 'Category_x'])).reset_index(drop=True)


# # Применение функции определения выбросов в пределах каждой группы бренд-категории
# grouped = merged_data.groupby(['brand_x', 'Category_x'], group_keys=False, as_index=False)
# merged_data = grouped.apply(lambda x: identify_outliers(x.drop(columns=['brand_x', 'Category_x']), 'price_difference_%')).reset_index(drop=True)



# # Применение функции определения выбросов в пределах каждой группы бренд-категории
# grouped = merged_data.groupby(['brand_x', 'Category_x'], group_keys=False, as_index=False)

# # Применение функции к каждой группе отдельно
# def apply_identify_outliers(group):
#     group = group.drop(columns=['brand_x', 'Category_x'])
#     return identify_outliers(group, 'price_difference_%')

# merged_data = grouped.apply(apply_identify_outliers).reset_index(drop=True)


# Добавление столбца с идентификатором группы
merged_data['group_id'] = merged_data.groupby(['brand_x', 'Category_x']).ngroup()

# Применение функции определения выбросов к каждой группе
merged_data = merged_data.groupby('group_id').apply(lambda x: identify_outliers(x, 'price_difference_%')).reset_index(drop=True)

# Удаление столбца group_id, так как он больше не нужен
merged_data.drop(columns=['group_id'], inplace=True)


#display(merged_data.head())

# Удаление столбца z_score, так как он больше не нужен
merged_data.drop(columns=['z_score'], inplace=True)

# Сохранение результата в новый файл Excel
merged_data.to_excel('data/merged_data_with_outliers.xlsx', index=False)

print("Данные с разницей цен и идентификацией выбросов сохранены в 'merged_data_with_outliers.xlsx'.")

,brand_x,Category_x,producer_id,producer_color,price,clean_producer_id,clean_producer_color,brand_y,Category_y,item_id,color_id,current price,clean_item_id,clean_color_id,price_difference_%
0,Valentino,Shoes,aaaaa1111_11,black,167,aaaaa111111,black,Valentino,Shoes,aaaaa111111,black,247,aaaaa111111,black,-32.388664
1,Valentino,Shoes,aaaaa1111_12,black,188,aaaaa111112,black,Valentino,Shoes,aaaaa111112,black,161,aaaaa111112,black,16.770186
2,Valentino,Shoes,aaaaa1111_13,black,184,aaaaa111113,black,Valentino,Shoes,aaaaa111113,black,234,aaaaa111113,black,-21.367521
3,Valentino,Shoes,aaaaa1111_14,bla//ck,196,aaaaa111114,black,Valentino,Shoes,aaaaa111114,black,167,aaaaa111114,black,17.365269
4,Valentino,Shoes,aaaaa1111_15,bla\\ck,250,aaaaa111115,black,Valentino,Shoes,aaaaa111115,black,153,aaaaa111115,black,63.398693


Данные с разницей цен и идентификацией выбросов сохранены в 'merged_data_with_outliers.xlsx'.


C:\Users\stepu\AppData\Local\Temp\ipykernel_198628\884222778.py:62: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_data = merged_data.groupby('group_id').apply(lambda x: identify_outliers(x, 'price_difference_%')).reset_index(drop=True)


In [ ]:
# Исправленный код для истранения - DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns.
import pandas as pd
import numpy as np
#import openpyxl

# Считать данные из Excel в DataFrame
data_parsing = pd.read_excel('data/Data_TSUM.xlsx', sheet_name='Data_Parsing')
data_company = pd.read_excel('data/Data_TSUM.xlsx', sheet_name='Data_Company')


# Функция для очистки ID от указанных служебных символов
def clean_id(id):
    return ''.join(e for e in id if e not in ['_', '-', '~', '\\', '/'])

# Очистка ID в обоих DataFrame
data_parsing['clean_producer_id'] = data_parsing['producer_id'].apply(clean_id)
data_parsing['clean_producer_color'] = data_parsing['producer_color'].apply(clean_id)
data_company['clean_item_id'] = data_company['item_id'].apply(clean_id)
data_company['clean_color_id'] = data_company['color_id'].apply(clean_id)

# Объединение DataFrame по очищенным ID
merged_data = pd.merge(data_parsing, data_company, left_on=['clean_producer_id', 'clean_producer_color'], right_on=['clean_item_id', 'clean_color_id'], how='left')

# Расчет разницы цен в процентах
merged_data['price_difference_%'] = ((merged_data['price'] - merged_data['current price']) / merged_data['current price']) * 100

#display(merged_data.head())

# Добавление столбца с идентификатором группы
merged_data['group_id'] = merged_data.groupby(['brand_x', 'Category_x']).ngroup()

#display(merged_data.head(10))

# Функция для вычисления z-оценок и определения выбросов больше 2 сигм
def calculate_z_scores(df, column):
    df['z_score'] = (df[column] - df[column].mean()) / df[column].std()
    df['outlier'] = np.where(np.abs(df['z_score']) > 2, 'Yes', 'No') # Используем порог z-оценки 2 для определения выбросов
    return df

# Применение функции к каждой группе
grouped = merged_data.groupby('group_id')
results = []

#display(grouped.head(10))

for name, group in grouped:
    group = group.drop(columns=['brand_x', 'Category_x'])
    result = calculate_z_scores(group, 'price_difference_%')
    results.append(result)

display(results[0].head())

# Объединение результатов обратно в один DataFrame
merged_data = pd.concat(results).reset_index(drop=True)

# Удаление столбца group_id и z_score, так как они больше не нужны
merged_data.drop(columns=['group_id', 'z_score'], inplace=True)

# Сохранение результата в новый файл Excel
merged_data.to_excel('data/merged_data_with_outliers.xlsx', index=False)

print("Данные с разницей цен и идентификацией выбросов сохранены в 'data/merged_data_with_outliers.xlsx'.")

#display(merged_data.head())


,producer_id,producer_color,price,clean_producer_id,clean_producer_color,brand_y,Category_y,item_id,color_id,current price,clean_item_id,clean_color_id,price_difference_%,group_id,z_score,outlier
39,bbbbb1111_25,~~~red,180,bbbbb111125,red,Brioni,Bags,bbbbb111125,red,153,bbbbb111125,red,17.647059,0,0.463783,No
40,bbbbb1111_26,~~~red,187,bbbbb111126,red,Brioni,Bags,bbbbb111126,red,229,bbbbb111126,red,-18.340611,0,-0.763806,No
41,bbbbb1111_27,~~~red,153,bbbbb111127,red,Brioni,Bags,bbbbb111127,red,183,bbbbb111127,red,-16.393443,0,-0.697386,No
42,bbbbb1111_28,~~~red,272,bbbbb111128,red,Brioni,Bags,bbbbb111128,red,204,bbbbb111128,red,33.333333,0,0.998863,No
43,bbbbb1111_29,~~~red,213,bbbbb111129,red,Brioni,Bags,bbbbb111129,red,197,bbbbb111129,red,8.121827,0,0.138864,No


Данные с разницей цен и идентификацией выбросов сохранены в 'data/merged_data_with_outliers.xlsx'.
